In [11]:
import os
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import sys 
sys.path.append(r'..\..\utils') 
from utils import get_seaad_colors, get_tx_order


In [2]:
# t_labels = meta_file = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\datasets\human_exc\data\Features\dendrite\metadata.csv', index_col='specimen_id')
# ttype_mapping_alg = 'hann' #hann, corr, tree

# me_labels = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\datasets\human_exc\analysis\20240911_seaad\MET_typing\config_tree_top_four\refined_labels.csv', index_col=0)
# me_labels = me_labels.rename(columns={'0':'me_type'})
# met_labels = pd.read_csv(r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\datasets\human_exc\analysis\20240911_seaad\MET_typing\config_tree_top_four\met_cell_assignments.csv', index_col='specimen_id')

# out_dir = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\datasets\human_exc\analysis\20240911_seaad\MET_typing\config'
# # out_dir = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\datasets\human_exc\analysis\20240911_seaad\MET_typing\config_tree_top_four'

# river_type = 't-met' #'t-me' 't-met' 'me-met'


In [3]:
met_dir = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\datasets\human_exc\analysis\20250815_seaad\MET_typing\config_v1'
data_dir = r'\\allen\programs\celltypes\workgroups\mousecelltypes\SarahWB\datasets\human_exc\analysis\20250815_seaad\MET_typing\data'


In [4]:
me_labels = pd.read_csv(os.path.join(met_dir, 'refined_labels.csv'), index_col=0)
me_labels = me_labels.rename(columns={'0':'me_type'})
met_labels = pd.read_csv(os.path.join(met_dir, 'met_cell_assignments.csv'), index_col=0)

t_labels = pd.read_csv(os.path.join(data_dir, 'metadata.csv'), index_col='spec_id_label')

river_type = 't-met' #'t-me' 't-met' 'me-met'


In [8]:
#HANN mapping
class_col = 'class_label_Hierarchical'
subclass_col = 'subclass_label_Hierarchical'
cluster_col = 'cluster_label'

t_labels = t_labels[[class_col, subclass_col, cluster_col]]

#combine the data 
print(f'{len(t_labels)} with t-type')
data = t_labels.merge(me_labels, how='inner', left_index=True, right_index=True)
print(f'{len(data)} with me-type')
data = data.merge(met_labels, how='inner', left_index=True, right_index=True)
print(f'{len(data)} with met-type')

data.me_type = data.me_type.apply(lambda x: f'ME_{x}')
data.met_type = data.met_type.apply(lambda x: f'MET_{x}')

240 with t-type
240 with me-type
233 with met-type


In [9]:
data

,class_label_Hierarchical,subclass_label_Hierarchical,cluster_label,me_type,met_type
569835804,Neuronal: Glutamatergic,L2/3 IT,L2/3 IT_1,ME_0,MET_0
571715966,Neuronal: Glutamatergic,L2/3 IT,L2/3 IT_1,ME_0,MET_0
595583062,Neuronal: Glutamatergic,L2/3 IT,L2/3 IT_1,ME_0,MET_0
596898838,Neuronal: Glutamatergic,L2/3 IT,L2/3 IT_1,ME_0,MET_0
602165685,Neuronal: Glutamatergic,L2/3 IT,L2/3 IT_1,ME_0,MET_0
...,...,...,...,...,...
1187969093,Neuronal: Glutamatergic,L6 IT Car3,L6 IT Car3_3,ME_10,MET_1
1220332582,Neuronal: Glutamatergic,L6 IT Car3,L6 IT Car3_3,ME_14,MET_10
1263331624,Neuronal: Glutamatergic,L5/6 NP,L5/6 NP_1,ME_20,MET_8
1162982233,Neuronal: Glutamatergic,L6 IT,L6 IT_1,ME_4,MET_6


In [12]:
ttype_order, subclass_order = get_tx_order()
color_dict = get_seaad_colors()
color_dict = dict(zip(color_dict.keys(), [c[:-2] for c in color_dict.values()]))

In [13]:
t_ordered_cluster = ttype_order

# me cluster order 
me_ordered_cluster = data['me_type'].unique()
me_ordered_cluster = sorted(me_ordered_cluster, key=lambda x: int(x.split('_')[1]))
me_ordered_cluster = list(me_ordered_cluster)

# met cluster order 
met_ordered_cluster = data['met_type'].unique()
met_ordered_cluster = sorted(met_ordered_cluster, key=lambda x: int(x.split('_')[1]))
met_ordered_cluster = list(met_ordered_cluster)


In [14]:
# Choose a colormap (e.g., 'viridis', 'plasma', 'coolwarm', etc.)
colormap = plt.get_cmap("viridis")

# Sample n colors evenly from the colormap
colors = [colormap(i / (len(me_ordered_cluster) - 1)) for i in range(len(me_ordered_cluster))]  # Get n colors

# Convert RGBA to hex and create a dictionary mapping each string to a color
me_color_dict = {cluster: '#' + ''.join(f'{int(c * 255):02x}' for c in colors[i][:3]) for i, cluster in enumerate(me_ordered_cluster)}


In [15]:
source_target_pairs = {
    't-me':[cluster_col, 'me_type', color_dict, t_ordered_cluster, me_ordered_cluster],
    't-met':[cluster_col, 'met_type', color_dict, t_ordered_cluster, met_ordered_cluster],
    'me-met':['me_type', 'met_type', me_color_dict, me_ordered_cluster, met_ordered_cluster],   
}

[source_col, target_col, node_colors, source_nodes, target_nodes ] = source_target_pairs[river_type]

In [16]:
source_nodes_sorted = [node for node in ttype_order if node in source_nodes]

In [17]:
t_ordered_cluster

['L2/3 IT_6',
 'L2/3 IT_1',
 'L2/3 IT_7',
 'L2/3 IT_8',
 'L2/3 IT_12',
 'L2/3 IT_10',
 'L2/3 IT_13',
 'L2/3 IT_5',
 'L2/3 IT_3',
 'L2/3 IT_2',
 'L4 IT_3',
 'L4 IT_4',
 'L4 IT_2',
 'L4 IT_1',
 'L5 IT_6',
 'L5 IT_5',
 'L5 IT_2',
 'L5 IT_1',
 'L5 IT_7',
 'L5 IT_3',
 'L5/6 NP_2',
 'L5/6 NP_6',
 'L5/6 NP_1',
 'L5/6 NP_3',
 'L5/6 NP_4',
 'L5 ET_1',
 'L5 ET_2',
 'L5 ET_ME',
 'L6 IT Car3_1',
 'L6 IT Car3_3',
 'L6 IT Car3_2',
 'L6 IT_2',
 'L6 IT_1',
 'L6 CT_3',
 'L6 CT_2',
 'L6 CT_4',
 'L6 CT_1',
 'L6b_6',
 'L6b_3',
 'L6b_5',
 'L6b_1',
 'L6b_4',
 'L6b_2']

In [18]:
data[cluster_col] = pd.Categorical(data[cluster_col], t_ordered_cluster)
data.sort_values([cluster_col], inplace = True)



In [19]:
data

,class_label_Hierarchical,subclass_label_Hierarchical,cluster_label,me_type,met_type
720878801,Neuronal: Glutamatergic,L2/3 IT,L2/3 IT_6,ME_7,MET_2
653854241,Neuronal: Glutamatergic,L2/3 IT,L2/3 IT_6,ME_1,MET_2
654180841,Neuronal: Glutamatergic,L2/3 IT,L2/3 IT_6,ME_2,MET_2
689306818,Neuronal: Glutamatergic,L2/3 IT,L2/3 IT_6,ME_1,MET_2
689360253,Neuronal: Glutamatergic,L2/3 IT,L2/3 IT_6,ME_0,MET_2
...,...,...,...,...,...
1272049036,Neuronal: Glutamatergic,L6 CT,L6 CT_1,ME_5,MET_7
1310107671,Neuronal: Glutamatergic,L6 CT,L6 CT_1,ME_16,MET_7
1163624259,Neuronal: Glutamatergic,L6 CT,L6 CT_1,ME_14,MET_7
1216089541,Neuronal: Glutamatergic,L6 CT,L6 CT_1,ME_16,MET_7


In [23]:
# Create nodes
# nodes = source_nodes + target_nodes
nodes = source_nodes_sorted + target_nodes

# Calculate x and y positions for nodes
x_positions = [0] * len(source_nodes_sorted) + [1] * len(target_nodes)  # x = 0 for source, x = 1 for target
y_positions = list(range(len(source_nodes_sorted))) + list(range(len(target_nodes)))  # Sequential y positions

# Normalize y positions to fit in [0, 1] (required by Plotly)
y_positions = [y / max(y_positions) for y in y_positions]

# Create links
links = []
for i in range(len(data)):
    source_idx = nodes.index(data[source_col].iloc[i])
    target_idx = nodes.index(data[target_col].iloc[i])
    links.append({source_col: source_idx, target_col: target_idx, 'value': 1, 'source_color': node_colors[data[source_col].iloc[i]]})

# Define Sankey plot
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="white", width=0.5),
        label=nodes,
        # color=[node_colors[node] for node in nodes]  # Assigning colors based on the node_colors dictionary
        color = [node_colors.get(node, '#808080') for node in nodes],
        x=x_positions,  # Explicit x positions
        y=y_positions  # Explicit y positions


    ),
    link=dict(
        source=[link[source_col] for link in links],
        target=[link[target_col] for link in links],
        value=[link['value'] for link in links],
        color=[link['source_color'] for link in links]  #NOTE uncomment to plot links as source node color

    ),
    arrangement="snap"
)])


# Update layout
fig.update_layout(title_text="{} to {}".format(source_col, target_col), font_size=10,width=300, height=1000)

# Save figure 
fig.write_html(os.path.join(met_dir, "{} to {}.html".format(source_col, target_col)))
fig.write_image(os.path.join(met_dir, "{} to {}.pdf".format(source_col, target_col)), format="pdf")

# Show plot
fig.show()


In [25]:
print(nodes)

['L2/3 IT_6', 'L2/3 IT_1', 'L2/3 IT_7', 'L2/3 IT_8', 'L2/3 IT_12', 'L2/3 IT_10', 'L2/3 IT_13', 'L2/3 IT_5', 'L2/3 IT_3', 'L2/3 IT_2', 'L4 IT_3', 'L4 IT_4', 'L4 IT_2', 'L4 IT_1', 'L5 IT_6', 'L5 IT_5', 'L5 IT_2', 'L5 IT_1', 'L5 IT_7', 'L5 IT_3', 'L5/6 NP_2', 'L5/6 NP_6', 'L5/6 NP_1', 'L5/6 NP_3', 'L5/6 NP_4', 'L5 ET_1', 'L5 ET_2', 'L5 ET_ME', 'L6 IT Car3_1', 'L6 IT Car3_3', 'L6 IT Car3_2', 'L6 IT_2', 'L6 IT_1', 'L6 CT_3', 'L6 CT_2', 'L6 CT_4', 'L6 CT_1', 'L6b_6', 'L6b_3', 'L6b_5', 'L6b_1', 'L6b_4', 'L6b_2', 'MET_0', 'MET_1', 'MET_2', 'MET_3', 'MET_4', 'MET_5', 'MET_6', 'MET_7', 'MET_8', 'MET_9', 'MET_10', 'MET_11', 'MET_12', 'MET_13', 'MET_14', 'MET_15', 'MET_16']
